### 라이브러리

In [ ]:
!pip install langchain-community
!pip install -U langchain-community
!pip install --upgrade langchain
!pip install tiktoken
!pip install langchain-google-genai
!pip install chromadb
!pip install langchain_huggingface
!pip install ragas
!pip install faiss-cpu
!pip install faiss-gpu

  Using cached langchain_huggingface-0.1.2-py3-none-any.whl.metadata (1.3 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-p

In [ ]:
import os
import re
import pandas as pd
import bs4
import tiktoken
from tqdm import tqdm
from collections import defaultdict
from IPython.display import clear_output
import time

from langchain.docstore.document import Document
from langchain_community.document_loaders import WebBaseLoader
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from ragas import evaluate
from ragas.metrics import context_precision, faithfulness

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate

### API Key

In [ ]:
# gemini
YOUR_API_KEY = ''
os.environ['GOOGLE_API_KEY'] = YOUR_API_KEY

### 데이터

In [ ]:
# CSV
file_path = "/content/drive/MyDrive/aiffel_final_project/data_renew/aiffel_book_preprocessing_1.csv"
df = pd.read_csv(file_path)

In [ ]:
df.columns

Index(['ISBN', 'ITEM_ID', 'BID', 'GOODS_NO', '분류', '제목', '원제', '저자', '발행자',
       '발행일', '페이지', '가격', '표지', '책소개', '저자소개', '목차', '출판사리뷰', 'INSERT_DATE',
       'UPDATE_DATE'],
      dtype='object')

In [ ]:
df = df.sample(n=10000, random_state=2025)

### Chunking

In [ ]:
# 텍스트 분할 함수 (null 값은 빈 문자열로 처리)
def split_text(text, chunk_size=1000, overlap=100):
    if text is None or pd.isnull(text):
        return [""]
    chunks = []
    for i in range(0, len(text), chunk_size - overlap):
        chunks.append(text[i:i + chunk_size])
    return chunks

In [ ]:
metadata_columns = ['ISBN', 'ITEM_ID', 'BID', 'GOODS_NO', '발행자', '발행일', '페이지', '가격', '표지', 'INSERT_DATE', 'UPDATE_DATE'] # 원제
vector_doc_columns = ['제목', '분류', '저자', '책 소개', '저자소개', '목차', '출판사리뷰']

In [ ]:
# 메타데이터에 포함될 컬럼과 벡터 DB Documents에 들어갈 핵심 데이터 컬럼 정의
# metadata_columns = ['발행자', '발행일', '페이지', '가격']
# vector_doc_columns = ['제목', '분류', '저자','저자소개', '책 소개', '목차', '출판사리뷰']

# RAG_DB 구성: 각 행의 vector_doc_columns를 하나의 텍스트로 합치고, metadata_columns에 해당하는 데이터는 별도 dict에 저장
RAG_DB = []
for index, row in df.iterrows():
    # 핵심 데이터(문서 내용) 생성: 각 컬럼명과 값을 줄바꿈 형태로 연결
    doc_text = ""
    for col in vector_doc_columns:
        value = row.get(col, "")
        if pd.isnull(value):
            value = ""
        doc_text += f"{col}: {value}\n"

    # 텍스트 분할 (문장이 길 경우 대비)
    chunks = split_text(doc_text)

    # 메타데이터 구성: metadata_columns에 있는 모든 데이터를 개별적으로 저장
    metadata = {}
    for col in metadata_columns:
        metadata[col] = row.get(col, None)

    # 분할된 각 청크를 RAG_DB에 추가
    for chunk in chunks:
        RAG_DB.append({
            'text': chunk,
            'metadata': metadata
        })

In [ ]:
# LangChain Document 생성: 각 RAG_DB 항목의 text와 metadata를 그대로 사용
from langchain.docstore.document import Document

documents = [
    Document(
        page_content=entry['text'],
        metadata=entry['metadata']
    ) for entry in RAG_DB
]

### 임베딩 및 벡터스토어 생성

In [ ]:
# 임베딩 모델 로드 (HuggingFace의 BGE-m3-ko)
hf_embeddings = HuggingFaceEmbeddings(model_name="dragonkue/BGE-m3-ko")

# 벡터스토어 생성을 위해 텍스트와 임베딩 쌍 구성
text_embedding_pairs = []  # (텍스트, 임베딩) 튜플 리스트
metadata_list = []

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
for doc in tqdm(documents, desc="Processing Documents", unit="document"):
    embedding = hf_embeddings.embed_query(doc.page_content)  # 청크 임베딩
    text_embedding_pairs.append((doc.page_content, embedding))
    metadata_list.append(doc.metadata)

# FAISS 벡터스토어 생성
vectorstore = FAISS.from_embeddings(
    text_embeddings=text_embedding_pairs,
    metadatas=metadata_list,
    embedding=hf_embeddings
)

Processing Documents: 100%|██████████| 28927/28927 [46:56<00:00, 10.27document/s]


### Retrieval

In [ ]:
# Gemini-1.5-flash
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.0)

# FAISS 벡터스토어 retriever 생성 top k : 5
dense_retriever = vectorstore.as_retriever(
    search_kwargs={"k": 5}
)

# RetrievalQA 체인 구성 - 검색 문서도 같이 반환
dpr_qa_chain = RetrievalQA.from_chain_type(
    llm=llm_gemini,
    retriever=dense_retriever,
    return_source_documents=True
)

### Generation

In [ ]:
# 대화 히스토리와 사용자 선호도 저장용
user_preferences = defaultdict(list)
log_history = []  # 전체 대화 로그 저장용


# 검색 쿼리 + 추가 질문 생성을 위한 메뉴얼 프롬프트
multi_turn_prompt = PromptTemplate.from_template("""
사용자와의 대화 히스토리는 다음과 같아.:

{history}

사용자의 마지막 질문은 다음과 같아.:
"{query}"

## role :

{impormation}
1. 사용자가 책을 찾는 이유를 아는가?

1) 사용자는 심심해서 그냥 책을 읽고 싶어함.
2) 사용자는 추천받은 책을 통해서 정보나 기술을 얻고 싶어함.
3) 사용자는 흥미, 취미 생활 등의 일환으로 서적을 찾고 싶어함.

2. 사용자가 찾고자 하는 책에 대한 정보를 얼만큼 알고 있는가?

1) 사용자는 찾고자하는 책에 대한 어떤 사전 지식도 없음.
2) 사용자는 특정하는 책은 없으나, 카테고리 or 작가 or 관련 책 이름을 말하며 비슷한 책을 추천받고 싶어함.
3) 사용자는 확고하게 찾고 싶은 책이 존재하며, 해당 책이 없다면 해당 책과 최대한 비슷한 책을 찾고 싶어함.

3. 사용자 취향이 어떤 카테고리에 적합한가?

1) 소설 (Fiction)
현대 문학
고전 문학
판타지
SF/공상 과학
미스터리/스릴러
로맨스/연애 소설
사회 비판/풍자 소설
심리/철학 소설
역사 소설
전쟁 소설

2) 경제/경영
기업 경영/전략
주식/재테크
마케팅/브랜딩
경제학 입문
창업/스타트업
노동/일의 철학

3) 자기계발 (Self-Development)
시간관리/생산성
멘탈 관리/자기계발
습관 형성
리더십/커뮤니케이션
자기 탐색/자아실현

4) 시/에세이 (Poetry & Essays)
철학적 에세이
문학적 에세이
감성 에세이
자전적 에세이
여행 에세이
시집

5) 인문/교양 (Humanities & Culture)
철학/사상
역사
사회/정치
심리학
종교/명상


6) 취미/실용 (Hobby & Practical)
요리/음식
운동/건강
DIY/핸드메이드
사진/영상
글쓰기/창작
음악/예술



7) 어린이/청소년 (Children & Young Adult)
그림책
초등 필독서
청소년 소설
과학/탐구
어린이 경제/교양


{if}
사용자와 이루어지는 대화는 위 {impormation}에 기반하여 사용자의 선호도를 특정할 수 있도록 이루어져야해. 만약 충분한 사용자 정보를 얻었다면, 벡터 DB 내에서 아래 {strategy}를 활용해서 검색을 진행해.

{strategy}
1. 대화를 실시간으로 종합하면서 검색을 수행할 확률을 0에서 1 사이로 평가해. 0.7 이상이면 검색을 진행해.
    - 예시 : 사용자가 찾고자하는 카테고리를 안다 : 점수 증가
    - 예시 : 사용자가 책을 찾는 목적을 안다 (자기계발,취미,정보습득 등) : 점수 증가
2. 검색 조건이 충족되었다면, 사용자의 이전 질문들과 답변들을 10~40 단어 사이로 요약해서 벡터 DB 검색에 적절한 최종 쿼리를 생성해.
3. 검색에 활용될 쿼리에는 사용자의 이전 질문들과 답변들 중 카테고리 등과 같은 "핵심 키워드 2~5개"를 반드시 포함시키도록 해.
4. 검색에는 사용자의 이전 질문들과 답변들을 "책 추천 기준"을 포함하여 요약해.
    - 예시: "SF 장르 중에서도 AI 관련 테마를 가진 최신 베스트셀러 추천 or 한국의 근현대사를 최대한 사실적으로 기술하고 있는 역사책 추천"
5. 검색에는 사용자의 이전 질문들과 답변들을 요약할 때 "추천 기준"도 함께 포함하고 고려해서 검색을 진행해.
    - 예시: "최신 AI 관련 베스트셀러 중에서 평점 4.5 이상인 도서"

{else}
- 사용자의 선호도를 정확히 파악하지 못해서 검색을 진행할 수 없다면, {impormation}과 {strategy}에 알맞게 검색 점수를 올리는 방향으로 적절한 질문을 1개 생성해.
    - 예시 : "(카테고리를 모른다.) : 네가 관심있어하는 도서 카테고리가 뭐야?"
    - 예시 : "(사용자의 정확한 목적을 모른다.) : 네가 찾고 있는 지식 도서는 초급자 용이야? 상급자 용이야?"
- 사용자가 너의 목적이 뚜렷한 질문에도 불구하고 중복되는 내용을 3번 이상 답변한다거나, 최종적으로 5번 이상의 대화를 나눴음에도 검색할 수 없다면, 적절한 검색이 이뤄질 수 없음을 언급하도록 해.

출력 형식 예시:
1. 검색 확률: 0.8
2. 검색 쿼리: "AI 철학 관련 최신 도서"
3. 추가 질문: "AI 철학 관련해서 어떤 주제가 궁금하세요?"
""")

search_query_chain = LLMChain(llm=llm_gemini, prompt=multi_turn_prompt)

<ipython-input-21-c94ea7d4a4ac>:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  search_query_chain = LLMChain(llm=llm_gemini, prompt=multi_turn_prompt)


In [ ]:
import re
# 추천 이유는 "출판사리뷰"를 우선, 없으면 "책 소개"를 사용하도록
def extract_field(text, field_name):
    # 각 필드는 "필드명: 내용" 형태로 되어 있음
    pattern = rf"{field_name}:\s*(.*)"
    match = re.search(pattern, text)
    return match.group(1).strip() if match else ""

MIN_INFO_LENGTH = 10  # 최소 정보 길이 기준 (필요에 따라 조정 가능)

def generate_answer(query):
    result = dpr_qa_chain.invoke(query)
    formatted_answers = []

    for doc in result['source_documents']:
        metadata = doc.metadata

        # 제목과 저자: metadata 우선, 없으면 page_content에서 추출
        title = metadata.get("제목") or extract_field(doc.page_content, "제목")
        author = metadata.get("저자") or extract_field(doc.page_content, "저자")

        # page_content에서 "출판사리뷰"와 "책 소개"를 추출
        publisher_review = extract_field(doc.page_content, "출판사리뷰")
        book_intro = extract_field(doc.page_content, "책 소개")

        # 두 정보 모두 존재하면 결합, 하나만 있으면 해당 정보 사용
        if publisher_review and book_intro:
            combined_info = publisher_review + "\n" + book_intro
        elif publisher_review:
            combined_info = publisher_review
        elif book_intro:
            combined_info = book_intro
        else:
            combined_info = ""

        # 결합된 정보가 충분하지 않으면 추천이유ㅜ없음으로
        if not combined_info or len(combined_info.strip()) < MIN_INFO_LENGTH:
            reason = "추천 이유 정보 없음"
        else:
            # LLM 프롬프트 작성: 정보가 충분하지 않으면 추천 이유 없음 으로
            reason_prompt = (
                f"다음 정보를 참고하여, 이 책이 추천되는 이유를 간결하고 명확하게 요약해줘. "
                f"책의 특징이나 강점을 중심으로 설명해주면 좋겠어. 만약 제공된 정보가 충분하지 않다면, '추천 이유 정보 없음'이라고 응답해줘.\n\n정보:\n{combined_info}"
            )
            reason_response = llm_gemini.invoke(reason_prompt)
            generated_reason = reason_response.text().strip()  # text() 메서드에서 문자열

            # 생성된 텍스트가 너무 짧거나 fallback 문구와 유사하면 추천이유없음으로
            if not generated_reason or len(generated_reason) < 10 or "추천 이유 정보 없음" in generated_reason:
                reason = "추천 이유 정보 없음"
            else:
                reason = generated_reason

        formatted = f"{title}\n{author}\n{reason}"
        formatted_answers.append(formatted)

    answer = "\n\n".join(formatted_answers)
    return answer, None

In [ ]:
# 사용자 선호도 카테고리화 함수
def categorize_preference(question, response):
    if "장르" in question or "어떤 책" in question:
        user_preferences["genre"].append(response)
    elif "작가" in question or "좋아하는 작가" in question:
        user_preferences["author"].append(response)
    elif "목적" in question or "이유" in question:
        user_preferences["purpose"].append(response)
    else:
        user_preferences["misc"].append(response)

In [ ]:
# 최종 검색 쿼리 생성을 위한 프롬프트
final_query_prompt = PromptTemplate.from_template("""
지금까지의 대화 내용을 바탕으로, 사용자의 선호도와 요청을 반영하여 검색에 적절한 최종 쿼리를 생성해.
1. 검색에 적절한 최종 쿼리의 형태를 생성하는데, 요약과 키워드 추출을 알맞게 진행해.
    - 예시 : 기술 서적 - "머신러닝 파이썬 초급자"
    - 예시 : 소설 서적 - "주인공이 운명이 내린 시련을 극복하고 끝내 꿈을 쟁취하는 소설 중 심리 묘사가 심도깊은 소설"
대화 내용:
{history}
기본 검색 쿼리: {fallback}
최종 검색 쿼리:
""")
final_query_chain = LLMChain(llm=llm_gemini, prompt=final_query_prompt)

def robust_parse_llm_response(response_text):
    """
    LLM 응답 텍스트에서 검색 확률, 검색 쿼리, 추가 질문을 추출.
    """
    cleaned_text = re.sub(r'\*\*', '', response_text)
    search_score = None
    search_query = None
    follow_up_question = ""

    score_match = re.search(r"검색\s*확률[:：]\s*([\d\.]+)", cleaned_text)
    if score_match:
        try:
            search_score = float(score_match.group(1))
        except Exception as e:
            print("검색 확률 파싱 에러:", e)

    query_match = re.search(r"검색\s*쿼리[:：]\s*(.*)", cleaned_text)
    if query_match:
        search_query = query_match.group(1).strip()
        if search_query.startswith('"') and search_query.endswith('"'):
            search_query = search_query[1:-1].strip()

    follow_match = re.search(r"추가\s*질문[:：]\s*(.*)", cleaned_text)
    if follow_match:
        follow_up_question = follow_match.group(1).strip()
        if follow_up_question in ["(필요 없음)", "(없음)", ""]:
            follow_up_question = ""
        if follow_up_question.startswith('"') and follow_up_question.endswith('"'):
            follow_up_question = follow_up_question[1:-1].strip()

    return search_score, search_query, follow_up_question

In [ ]:
def search_and_generate_answer(query, query_history):
    while True:
        query_summary = "\n".join(query_history[-5:])  # 최근 5개 대화 요약
        search_decision_dict = search_query_chain.invoke({
            "history": query_summary,
            "query": query,
            "if": "✅ 검색이 가능한 경우:",
            "else": "❌ 아직 검색이 불가능한 경우:",
            "impormation": "📌 사용자 선호도 분석:",
            "strategy": "🔍 검색 전략:",
        })
        response_text = search_decision_dict["text"].strip()
        print("\n[🔍 LLM 응답 확인]\n", response_text)

        # LLM 응답 파싱
        search_score, base_search_query, follow_up_question = robust_parse_llm_response(response_text)
        print(f"\n[디버그] 파싱 결과: 검색 확률={search_score}, 기본 검색 쿼리='{base_search_query}', 추가 질문='{follow_up_question}'")

        # 파싱 실패 시 추가 정보 요청
        if search_score is None:
            print("\n[❌ LLM 응답 파싱 실패: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

        # 검색 확률이 충분(≥0.7)이고 기본 검색 쿼리가 있다면 최종 검색 쿼리 생성 후 DB 검색 진행
        if search_score >= 0.7 and base_search_query:
            final_search_query = final_query_chain.invoke({
                "history": "\n".join(query_history),
                "fallback": base_search_query
            })["text"].strip()
            print(f"\n[🔎 최종 검색 쿼리 생성]: {final_search_query}")

            answer, sources = generate_answer(final_search_query)
            if sources:
                book_info = "\n".join([f"- {title}" for title in sources])
                answer_with_info = f"{answer}\n\n[📚 책 정보]\n{book_info}"
                print("\n[📚 책 정보]\n", book_info)
            else:
                answer_with_info = answer
            return answer_with_info

        # 검색 확률이 낮거나 기본 검색 쿼리가 없으면 보충 질문 진행
        if follow_up_question:
            print(f"\n[🤖 보충 질문: {follow_up_question}]")
            query_history.append(f"AI: {follow_up_question}")
            user_response = input("\n사용자 응답: ")
            query_history.append(f"사용자: {user_response}")
            categorize_preference(follow_up_question, user_response)
            print("\n[📚 사용자 선호도 업데이트 완료!]")
            query = f"{query} {follow_up_question} {user_response}"
            continue

        if search_score < 0.7 or not base_search_query:
            print("\n[❌ 검색 확률 낮거나 검색 쿼리 없음: 추가 정보 필요]")
            extra_info = input("추가 정보를 입력해주세요: ")
            query_history.append(f"사용자(추가): {extra_info}")
            query = f"{query} {extra_info}"
            continue

In [ ]:
def interactive_multi_turn_qa():
    query_history = []  # 각 실행마다 초기화

    while True:
        clear_output(wait=True)
        print("📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)")
        print("-" * 50)

        query = input("질문을 입력하세요: ")

        if query.lower() == 'quit':
            print("\n[📝 대화 저장 중...]")
            log_history.append(query_history)
            print("대화가 저장되었습니다. 프로그램을 종료합니다.")
            break

        query_history.append(f"사용자: {query}")
        answer = search_and_generate_answer(query, query_history)

        print("\n[💬 AI의 답변]")
        print(answer)

        query_history.append(f"AI: {answer}")

        input("\n-> 계속하려면 Enter를 누르세요...")

In [ ]:
# 로그 저장
def show_log_history():
    print("\n[ 전체 대화 로그]")
    for i, session in enumerate(log_history, 1):
        print(f"\n 대화 세션 {i}:\n")
        print("\n".join(session))
        print("-" * 50)

In [ ]:
# 실행
interactive_multi_turn_qa()

📚 멀티턴 AI 기반 책 추천 시스템 (종료하려면 'quit' 입력)
--------------------------------------------------
질문을 입력하세요: 소설책 추천 좀

[🔍 LLM 응답 확인]
 1. 검색 확률: 0.2
2. 검색 쿼리:  (아직 생성 불가능)
3. 추가 질문: 어떤 종류의 소설을 좋아하세요? (예: 판타지, 로맨스, 스릴러, 현대소설, 고전소설 등)  혹은 어떤 분위기의 소설을 읽고 싶으세요? (예: 밝고 유쾌한, 어둡고 심오한, 잔잔하고 감동적인 등)

[디버그] 파싱 결과: 검색 확률=0.2, 기본 검색 쿼리='(아직 생성 불가능)', 추가 질문='어떤 종류의 소설을 좋아하세요? (예: 판타지, 로맨스, 스릴러, 현대소설, 고전소설 등)  혹은 어떤 분위기의 소설을 읽고 싶으세요? (예: 밝고 유쾌한, 어둡고 심오한, 잔잔하고 감동적인 등)'

[🤖 보충 질문: 어떤 종류의 소설을 좋아하세요? (예: 판타지, 로맨스, 스릴러, 현대소설, 고전소설 등)  혹은 어떤 분위기의 소설을 읽고 싶으세요? (예: 밝고 유쾌한, 어둡고 심오한, 잔잔하고 감동적인 등)]

사용자 응답: 판타지. 어둡고 심오한 판타지.

[📚 사용자 선호도 업데이트 완료!]

[🔍 LLM 응답 확인]
 1. 검색 확률: 0.8
2. 검색 쿼리: "어둡고 심오한 분위기의 판타지 소설 추천"
3. 추가 질문:  없음 (추가 질문이 필요 없다고 판단됨)


**설명:**

📌 사용자 선호도 분석:

1. 사용자가 책을 찾는 이유: 3) 사용자는 흥미, 취미 생활 등의 일환으로 서적을 찾고 싶어함. (단순히 심심해서가 아닌, 특정 분위기의 판타지 소설을 원한다는 점에서 추론 가능)

2. 사용자가 찾고자 하는 책에 대한 정보: 2) 사용자는 특정하는 책은 없으나, 카테고리(판타지) 및 분위기(어둡고 심오한)를 말하며 비슷한 책을 추천받고 싶어함.

3. 사용자 취향: 1) 소설 (Fiction) - 판타지


🔍 검색 전략 적용:

사용자의 선호도

KeyboardInterrupt: Interrupted by user